In [ ]:
import pandas as pd
import requests 
import json
from datetime import datetime, timezone, timedelta
import time

In [ ]:
df1 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\HeatSinkOutput.csv")
df2 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\MagStabOutput.csv")
df3 = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\webcrawler\GyroStabOutput.csv")

df_merge = pd.concat([df1, df2], ignore_index=True)
df_merge = pd.concat([df_merge, df3], ignore_index=True)
df_merge

In [ ]:
url = "https://esi.evetech.net/latest/universe/regions/"

response = requests.get(url)
region_ids = response.json()
print(f"Regions to scan: {len(region_ids)}")

In [ ]:
contracts = []
for region_id in region_ids:
    total_pages = 1
    current_page = 1
    base_url = "https://esi.evetech.net/latest/contracts/public/{}/".format(region_id)
    params = {"datasource": "tranquility", "page": current_page}
    
    while params["page"] <= total_pages:
        print(base_url)
        print(params)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)
        if total_pages == 1:
            total_pages = int(response.headers.get("X-Pages"))
            
        if response.status_code == 200:
            raw_contracts = response.json()
            item_exchange_contracts = [contract for contract in raw_contracts if contract["type"] == "item_exchange"]
            contracts.extend(item_exchange_contracts)
            params["page"] += 1
        else:
            raise ValueError("Unexpected API Response")
        if "ETag" not in params:
            params["ETag"] = response.headers.get("ETag").strip('\"')    
        

pd.json_normalize(contracts).to_parquet("./api_database/contracts.parquet")

In [ ]:
df_contracts = pd.read_parquet("./api_database/contracts.parquet")[["contract_id", "date_issued"]]

known_abyssal_contracts = pd.read_parquet("./api_database/item_stats.parquet")
known_abyssal_contracts_agg = known_abyssal_contracts.groupby("contract_id", as_index=False).agg({
    "item_id": "nunique"
}).rename(columns={"item_id":"abyssal_count"})

df_contracts = df_contracts.merge(known_abyssal_contracts_agg,how="left", on ="contract_id")
df_contracts["date_issued"] = pd.to_datetime(df_contracts["date_issued"])
df_contracts = df_contracts[(df_contracts["date_issued"] > (pd.to_datetime(datetime.utcnow() - timedelta(days=1)).tz_localize("UTC"))) | (df_contracts["abyssal_count"] > 0)][["contract_id"]]

last_run_items = pd.read_parquet("./api_database/contract_items.parquet")
last_run_contracts = last_run_items.groupby("contract_id", as_index=False).agg({
    "updated_at": "max"
})

contracts_to_scan = df_contracts.merge(last_run_contracts, how="left", on="contract_id")
contracts_to_scan = contracts_to_scan[(contracts_to_scan["updated_at"] < (pd.to_datetime("today") + pd.DateOffset(hours=-12))) | (contracts_to_scan["updated_at"].isna())] ["contract_id"].drop_duplicates(keep="first").tolist()

last_run_items = last_run_items.merge(df_contracts[["contract_id"]].drop_duplicates(keep="first"), how="inner", on="contract_id")
contracts_scanned = last_run_items[last_run_items["updated_at"] > (pd.to_datetime("today") + pd.DateOffset(hours=-12))]

In [ ]:
len(contracts_to_scan)

In [ ]:
items = []
contract_count = 0
print (len(contracts_to_scan))
for contract_id in contracts_to_scan:
    total_pages = 1
    page = 1
    while page <= total_pages:
        contract_count += 1
        base_url = "https://esi.evetech.net/latest/contracts/public/items/{}/".format(contract_id)
        params = {"datasource": "tranquility", "page": page}
                
        print(base_url)
        print(params)
        print(contract_count)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)
        if (total_pages == 1) & (response.headers.get("X-Pages") != None):
            total_pages = int(response.headers.get("X-Pages"))
            print(total_pages)
        
        if response.status_code == 200:
            if response.text != "":
                raw_contract_items = response.json()
                contract_items = [contract_item for contract_item in raw_contract_items]
                print(f"Items found: {len(contract_items)}")
                for contract_item in contract_items:
                    contract_item["contract_id"] = contract_id
                    contract_item["updated_at"] = datetime.utcnow()
                items.extend(contract_items)
        else:
            if int(response.headers.get("X-Esi-Error-Limit-Remain", 100)) < 20:
                break
            elif int(response.headers.get("X-Esi-Error-Limit-Remain", 50)) < 50:
                time.sleep(60)
        page += 1

contracts_scanned = pd.concat([contracts_scanned, pd.json_normalize(items)],ignore_index=True)
contracts_scanned.to_parquet("./api_database/contract_items.parquet")


In [ ]:
known_abyssal_contracts = pd.read_parquet("./api_database/item_stats.parquet")
known_abyssal_contracts_agg = known_abyssal_contracts.groupby(["contract_id"], as_index=False).agg({
    "item_id": "nunique"
}).rename(columns={"item_id":"item_id_count"})

# Clean up affteragg data TODO
contract_items = pd.read_parquet("./api_database/contract_items.parquet")
contract_items = contract_items.merge(known_abyssal_contracts_agg, how="left", on=["contract_id"])
contract_items_to_scan = contract_items[(contract_items["item_id_count"].isna()) & (contract_items["item_id"].notna())]
contract_items_to_scan["item_id"] = contract_items_to_scan["item_id"].astype("int64")
contract_items_to_scan["item_idtype_id"] = contract_items_to_scan["type_id"].astype("int64")

contract_items_scanned = contract_items[(contract_items["item_id_count"].notna())]
# contract_items_scanned = known_abyssal_contracts.merge(contract_items_scanned[["contract_id"]].drop_duplicates(keep="first"), how="inner", on="contract_id")
contract_items_scanned

In [ ]:
item_stats = []

for index, row in contract_items_to_scan.iterrows():
    if row["type_id"] in [49726, 49722, 49730]:
        base_url = f"https://esi.evetech.net/latest/dogma/dynamic/items/{row['type_id']}/{row['item_id']}/"
        params = {"datasource": "tranquility"}
        print(base_url)
        print(params)
        response = requests.get(base_url, params=params)
        print(response.headers)
        print(response)

        if response.status_code == 200:
            row_info = response.json()
            dogma_details = []
            dogma_details = [mod for mod in row_info["dogma_attributes"] if mod["attribute_id"] in [50,64,204]]
            for mod in dogma_details:
                mod["contract_id"] = row["contract_id"]
                mod["type_id"] = row["type_id"]
                mod["item_id"] = row["item_id"]
            item_stats.extend(dogma_details)
        else:
            if int(response.headers["X-Esi-Error-Limit-Remain"]) < 20:
                break
            elif int(response.headers["X-Esi-Error-Limit-Remain"]) < 50:
                time.sleep(60)
                
df_pivoted = pd.json_normalize(item_stats).pivot(index=['contract_id', 'type_id', 'item_id'], columns='attribute_id', values='value').reset_index()
df_pivoted = df_pivoted.rename_axis(None, axis=1)
df_pivoted = df_pivoted.rename(columns={50: "CPU", 64: "Damage" , 204:"ROF"})
output = pd.concat([contract_items_scanned, df_pivoted], ignore_index=True).drop_duplicates(keep="first")
 		
output.to_parquet("./api_database/item_stats.parquet")
output


In [ ]:
ship_dict = {"paladin":
        {"damage" :{
            "guns": 3.6
            ,"ammo": 70.8
            ,"hull": 2
            ,"weapon": 1.25
            ,"surgical_strike": 1.15
            ,"spec": 1.1
            ,"mar" : 1.25
            ,"LE-1006": 1.06
            }
        ,"rof": {
            "guns": 7.875
            ,"gunnery": 0.9
            ,"rapid_fire": 0.8
            ,"RF-906": 0.94
            ,"rig": 0.85
            }
        }
    }
for ship in ship_dict:
    dps = 1.0
    rof = 1.0
    for key, value in ship_dict[ship]["damage"].items():
        dps = dps * ship_dict[ship]["damage"][key]  
    for key, value in ship_dict[ship]["rof"].items():
        rof = rof * ship_dict[ship]["rof"][key]

In [ ]:
dps

In [ ]:
import pandas as pd
from itertools import combinations
import sys
from more_itertools import chunked
import os
from reference import MODS, COLUMNS, SHIPS, start_logging, price_df_norm
import datetime
import logging
import numpy as np
import math

start_logging('./info_log/matching_mods.log', __name__)
logger = logging.getLogger(__name__)
logger.info('Script started at %s', datetime.datetime.now())

Files = MODS

# if len(sys.argv) > 1:
#     if sys.argv[1] == "HeatSink":
#         Files = [MODS[0]]
#     elif sys.argv[1] == "MagStab":
#         Files = [MODS[1]]
#     elif sys.argv[1] == "GyroStab":
#         Files = [MODS[2]]
# else:
#     Files = MODS

chunkcount = 0


def stacking_penalty(u):
    result = math.exp(-(u/2.67)**2)
    return result

stacking_penalty_1 = stacking_penalty(1)
stacking_penalty_2 = stacking_penalty(2)
stacking_penalty_3 = stacking_penalty(3)
stacking_penalty_4 = stacking_penalty(4)

for Output in Files:
    combination_data = pd.read_csv(f".\webcrawler\{Output[0]}Output.csv") 
    filepath = "./sets/"+ Output[0] +"_sets.csv"
    
    if os.path.exists(filepath):
        os.remove(filepath)
        logger.info("Outdated file removed")
    logger.info("Creating new file") 

    combination_data = price_df_norm(combination_data)
    # combination_data["ROF"] = ((combination_data["ROF"] - 1) * 100).abs()
    data = combination_data[(combination_data["DPS"] >= 26) & (combination_data["Price"] <= 1000000000)]

    combination_data_list = list(data["ID"])
    combination_data_list = combination_data_list
    logger.info("combination_data list created")
    combination_data_list2 = list(combinations(combination_data_list, (4)))
        
    size = 3000000
    chunked_list = list(chunked(combination_data_list2, size))
    logger.info(f"chunked_list size: {size}")
    logger.info(f"chunked_list len: {len(chunked_list)}")
    data = data.astype({
        "Damage": float,
        "ROF": float,
        "CPU": float,
        "Contract": str
        })

    data = data[["ID", "CPU", 'Damage', 'ROF', "DPS", "Contract"]]

    chunkcount = 0

    ship_stats = SHIPS[Output[0]]
  
    for ship in ship_stats:
        dps = 1.0
        rof = 1.0
        for key, value in ship_stats["damage"].items():
            dps = dps * ship_stats["damage"][key]

        for key, value in ship_stats["rof"].items():
            rof = rof * ship_stats["rof"][key]
        

    
    for chunk in chunked_list:
        df = pd.DataFrame(chunk)

        data2 = df.merge(data.add_suffix('_first'), left_on=0,right_on="ID_first",how="left")
        data2 = data2.merge(data.add_suffix('_second'), left_on=1,right_on="ID_second",how="left")
        data2 = data2.merge(data.add_suffix('_third'), left_on=2,right_on="ID_third",how="left")
        data2 = data2.merge(data.add_suffix('_fourth'), left_on=3,right_on="ID_fourth",how="left")

        data2damage = data2[["Damage_first","Damage_second","Damage_third","Damage_fourth"]]
        data2damage = data2damage.rank(method="first", axis=1, ascending=False)
        data2damage = data2damage.replace({2:stacking_penalty_1, 3:stacking_penalty_2, 4:stacking_penalty_3})

        data2rof = data2[["ROF_first","ROF_second","ROF_third","ROF_fourth"]]
        data2rof = data2rof.rank(method="first", axis=1, ascending=True)
        data2rof = data2rof.replace({1:stacking_penalty_1, 2:stacking_penalty_2, 3:stacking_penalty_3, 4:stacking_penalty_4})

        data3 = data2.merge(data2damage.add_suffix('_damagaPenalty'),how="left",left_index=True, right_index=True)
        data3 = data3.merge(data2rof.add_suffix('_rofPenalty'),how="left",left_index=True, right_index=True)
        data3["raw_damage"] = dps
        for col in ["Damage_first", "Damage_second", "Damage_third", "Damage_fourth"]:
            data3[col] = ((data3[col] - 1) * data3[f"{col}_damagaPenalty"]) + 1 
            data3["raw_damage"] = data3["raw_damage"] * data3[col]

        data3["raw_rof"] = rof
        for col in ["ROF_first","ROF_second","ROF_third","ROF_fourth"]:
            data3[col] = 1 - ((1 - data3[col]) * data3[f"{col}_rofPenalty"])
            data3["raw_rof"] = data3["raw_rof"] * data3[col]

        data3["Total Damage"] = ((data3["raw_damage"] * 4) / (data3["raw_rof"]/1000)) * 2
        
        data3["TotalCPU"] = data3["CPU_first"] + data3["CPU_second"] + data3["CPU_third"] + data3["CPU_fourth"] 
        
        if Output[1] == "Market HeatSinks":       
            # Paladin
            # report2 = data3[(data3["TotalDamage"] >= 4.355) & (data3["TotalROF"] >= 33.11)].copy()
            report2 = data3[(data3["Total Damage"] >= 3350)].copy()
        elif Output[1] == "Market GyroStabs": 
            # Vargur 
            # report2 = data3[(data3["TotalDamage"] >= 4.369) & (data3["TotalROF"] >= 33.1114)].copy()
            report2 = data3[(data3["Total Damage"] >= 3350)].copy()
        elif Output[1] == "Market MagStabs": 
            # Kronos
            # report2 = data3[(data3["TotalDamage"] >= 4.368) & (data3["TotalROF"] >= 31.23)].copy()
            report2 = data3[(data3["Total Damage"] >= 4100)].copy()

        report3 = report2[[0,1,2,3,"Total Damage"]]     
  
        report2.to_csv(filepath, mode='a', index=False, header=True)
        chunkcount = chunkcount+1
        logger.info(f"{Output[0]} chunk " + str(chunkcount) + " saved.")    
        
    logger.info(f"All {Output[0]} chunks saved.")
    
logger.info('Script completed at %s', datetime.datetime.now())

In [ ]:
# df =pd.read_csv("D:\GitHub\Incursion-Damage-Mods\sets\HeatSink_sets.csv")
df[["0","1","2","3","TOTAL DAMAGE"]].sort_values(by="TOTAL DAMAGE", ascending=True)

In [ ]:
pd.read_csv("D:\GitHub\Incursion-Damage-Mods\sets\MagStab_sets.csv")[["0","1","2","3","TOTAL DAMAGE"]].sort_values(by="TOTAL DAMAGE", ascending=True)

In [1]:
import pandas as pd
df = pd.read_csv("D:\GitHub\Incursion-Damage-Mods\sets\GyroStab_sets.csv",low_memory=False)
df

In [ ]:
data3[(data3[0] == "https://mutaplasmid.space/module/1040887384497/") & (data3[1] == "https://mutaplasmid.space/module/1037739435504/") & (data3[2] == "https://mutaplasmid.space/module/1043429418844/")& (data3[3] == "https://mutaplasmid.space/module/1043508303551/")][["ROF_first","ROF_second","ROF_third","ROF_fourth", 'ROF_first_rofPenalty',       'ROF_second_rofPenalty',
              'ROF_third_rofPenalty',       'ROF_fourth_rofPenalty',]]